<a href="https://colab.research.google.com/github/camily-a/SI-not-2021/blob/main/SistemaPerfilSocioeconimico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

----- INSTRUÇÕES ----------
- Insira o documento de dados (do tipo .xlsx) na área de arquivos ao lado esquerdo.

- Clique no botão de executar código.

- Insira o nome do arquivo adicionado(com .xlsx).


In [15]:
import pandas as pd #apelido da biblioteca
from datetime import date 
import sys
import plotly.express as px # importa a biblioteca

name = input('Insira o nome do arquivo de dados (inclui .xlsx) : ')
tabela = pd.read_excel(name) #importa dados 

################

# organiza tabela por data de envio
tabela = tabela.sort_values(by='Carimbo de data/hora', ascending=True)
#exclui linhas duplicadas mantendo a última enviada
tabela = tabela.drop_duplicates(subset=["Informe os 7 últimos dígitos do seu RA (109048xxxxxxx) "], keep="last") 

# exclui colunas irrelevantes para a Análise
tabela = tabela.drop("Carimbo de data/hora", axis=1) # axis 1 p/ coluna e 0 para linha
tabela = tabela.drop("Informe os 7 últimos dígitos do seu RA (109048xxxxxxx) ", axis=1)

# mostra todas as linhas da tabela
pd.set_option("max_rows", None)

#################

# trocando valores vazios
tabela.fillna("Preferiu não responder", inplace=True)

#############

# convertendo para string e padronizando (strip , lower, upper) 
# pessoas no domicilio

tabela["Quantas pessoas, incluindo você, moram no seu domicílio?"] = tabela["Quantas pessoas, incluindo você, moram no seu domicílio?"].astype("string")
tabela["Quantas pessoas, incluindo você, moram no seu domicílio?"] = tabela["Quantas pessoas, incluindo você, moram no seu domicílio?"].str.strip()
tabela["Quantas pessoas, incluindo você, moram no seu domicílio?"] = tabela["Quantas pessoas, incluindo você, moram no seu domicílio?"].str.lower()

# tempo de moradia 
tabela['Tempo de moradia neste domicílio.'] = tabela['Tempo de moradia neste domicílio.'].astype("string")
tabela['Tempo de moradia neste domicílio.'] = tabela['Tempo de moradia neste domicílio.'].str.strip()
tabela['Tempo de moradia neste domicílio.'] = tabela['Tempo de moradia neste domicílio.'].str.lower()

# empresa que esta contratado
tabela["Qual a empresa que você está contratado agora?"] = tabela["Qual a empresa que você está contratado agora?"].astype("string")
tabela["Qual a empresa que você está contratado agora?"] = tabela["Qual a empresa que você está contratado agora?"].str.strip()
tabela["Qual a empresa que você está contratado agora?"] = tabela["Qual a empresa que você está contratado agora?"].str.upper()



#################

# Descobrindo idade a partir da data de nascimento

data_nasc = (tabela["Qual a sua data de nascimento?"])
days_in_year = 365.25 
agora = pd.Timestamp('today')
 
idade = (agora - data_nasc).dt.days / (days_in_year)
idade = idade.astype(int)

tabela["Qual a sua data de nascimento?"] = idade
tabela.rename(columns={"Qual a sua data de nascimento?": 'Idade'}, inplace = True)

#########
## tempo de moradia (caso = desde que nasci)

# coloca os index em sequencia novamente (reseta index)
tabela.reset_index(drop=True, inplace=True)

tempo = (tabela.index[(tabela['Tempo de moradia neste domicílio.'] == 'desde que nasci')].tolist())

tempo = tabela.iloc[tempo]['Idade'].tolist()

tabela["Tempo de moradia neste domicílio."] = tabela["Tempo de moradia neste domicílio."].replace('desde que nasci', tempo[0])


#display(tabela["Tempo de moradia neste domicílio."])                                                                                                 
############

############

# agrupando idades


tabela['Idade']=pd.cut(
    tabela['Idade'],
    bins=[-1, 20, 30, 40, sys.maxsize],
    labels=['Até 20 anos', 'Entre 21 e 30 anos', 'Entre 31 e 40 anos', 'Mais de 40 anos']
 )

tabela.rename(columns={"Idade": 'Faixa Etária'}, inplace = True)


########### 


# padronizando valores iguais , escritos de forma diferente (substituindo)

# pessoas no domicilio

tabela["Quantas pessoas, incluindo você, moram no seu domicílio?"].replace(regex=['dez', 
                                                                        '10'], value=10, inplace = True)
tabela["Quantas pessoas, incluindo você, moram no seu domicílio?"].replace(regex=['09','nove', 
                                                                        '9'], value=9, inplace = True)
tabela["Quantas pessoas, incluindo você, moram no seu domicílio?"].replace(regex=['08','oito', 
                                                                        '8'], value=8, inplace = True)
tabela["Quantas pessoas, incluindo você, moram no seu domicílio?"].replace(regex=['07','sete', 
                                                                        '7'], value=7, inplace = True)
tabela["Quantas pessoas, incluindo você, moram no seu domicílio?"].replace(regex=['06','seis', 
                                                                        '6'], value=6, inplace = True)
tabela["Quantas pessoas, incluindo você, moram no seu domicílio?"].replace(regex=['05','cinco', 
                                                                         '5'], value=5, inplace = True)
tabela["Quantas pessoas, incluindo você, moram no seu domicílio?"].replace(regex=['04','quatro', 
                                                                          '4'], value=4, inplace = True)
tabela["Quantas pessoas, incluindo você, moram no seu domicílio?"].replace(regex=['03','três','tres' 
                                                                          '3'], value=3, inplace = True)
tabela["Quantas pessoas, incluindo você, moram no seu domicílio?"].replace(regex=['02','duas', 'dois'
                                                                          '2'], value=2, inplace = True)                                                                                                                                                 
tabela["Quantas pessoas, incluindo você, moram no seu domicílio?"].replace(regex=['01','uma','um'
                                                                           '1'], value=1, inplace = True) 
              

# empresa que esta contratado 

tabela["Qual a empresa que você está contratado agora?"] = tabela["Qual a empresa que você está contratado agora?"].replace(regex=['PREFERIU NÃO RESPONDER','NENHUM',
                                                                                                                              'NENHUMA','NÃO'], value='NÃO TRABALHO')

                                                          
# tempo de moradia

tabela["Tempo de moradia neste domicílio."].replace(regex=['mes','meses','semana','semanas','dias','dia'],
                                                               value = 0.9, inplace = True)
tabela["Tempo de moradia neste domicílio."].replace(regex=['30', 'trinta'],
                                                              value = 30, inplace = True)
tabela["Tempo de moradia neste domicílio."].replace(regex=['29' ,'vinte e nove'],
                                                              value = 29, inplace = True)
tabela["Tempo de moradia neste domicílio."].replace(regex=['28' ,'vinte e oito'],
                                                              value = 28, inplace = True)
tabela["Tempo de moradia neste domicílio."].replace(regex=['27', 'vinte e sete'],
                                                              value = 27, inplace = True)
tabela["Tempo de moradia neste domicílio."].replace(regex=['26' ,'vinte e seis'],
                                                              value = 26, inplace = True)
tabela["Tempo de moradia neste domicílio."].replace(regex=['25' ,'vinte e cinco'],
                                                              value = 25, inplace = True)
tabela["Tempo de moradia neste domicílio."].replace(regex=['24', 'vinte e quatro'],
                                                           value = 24, inplace = True)
tabela["Tempo de moradia neste domicílio."].replace(regex=['23', 'vinte e três', 'vinte e tres'],
                                                              value = 23, inplace = True)
tabela["Tempo de moradia neste domicílio."].replace(regex=['22' ,'vinte e dois'],
                                                              value = 22, inplace = True)
tabela["Tempo de moradia neste domicílio."].replace(regex=['21' ,'vinte e um'],
                                                              value = 21, inplace = True)
tabela["Tempo de moradia neste domicílio."].replace(regex=['20', 'vinte'],
                                                              value = 20, inplace = True)
tabela["Tempo de moradia neste domicílio."].replace(regex=['19' ,'dezenove'],
                                                              value = 19, inplace = True)
tabela["Tempo de moradia neste domicílio."].replace(regex=['18', 'dezoito'],
                                                              value = 18, inplace = True)
tabela["Tempo de moradia neste domicílio."].replace(regex=['17', 'dezessete'],
                                                              value = 17, inplace = True)
tabela["Tempo de moradia neste domicílio."].replace(regex=['16','dezesseis'],
                                                              value = 16, inplace = True)
tabela["Tempo de moradia neste domicílio."].replace(regex=['15','quinze'],
                                                              value = 15, inplace = True)
tabela["Tempo de moradia neste domicílio."].replace(regex=['14','quatorze'],
                                                              value = 14, inplace = True)
tabela["Tempo de moradia neste domicílio."].replace(regex=['13','treze'],
                                                              value = 13, inplace = True)
tabela["Tempo de moradia neste domicílio."].replace(regex=['12','doze'],
                                                              value = 12, inplace = True)
tabela["Tempo de moradia neste domicílio."].replace(regex=['11','onze'],
                                                              value = 11, inplace = True)
tabela["Tempo de moradia neste domicílio."].replace(regex=['10','dez'],
                                                              value = 10, inplace = True)
tabela["Tempo de moradia neste domicílio."].replace(regex=['09','9','nove'],
                                                              value = 9, inplace = True)
tabela["Tempo de moradia neste domicílio."].replace(regex=['08','8','oito'],
                                                              value = 8, inplace = True)
tabela["Tempo de moradia neste domicílio."].replace(regex=['07','7','sete'],
                                                              value = 7, inplace = True)
tabela["Tempo de moradia neste domicílio."].replace(regex=['06','6','seis'],
                                                               value = 6, inplace = True)
tabela["Tempo de moradia neste domicílio."].replace(regex=['05','5','cinco'],
                                                             value = 5, inplace = True)
tabela["Tempo de moradia neste domicílio."].replace(regex=['04','4','quatro'],
                                                             value = 4, inplace = True)
tabela["Tempo de moradia neste domicílio."].replace(regex=['03','3','tres','três'],
                                                              value = 3, inplace = True)
tabela["Tempo de moradia neste domicílio."].replace(regex=['02','2','dois'],
                                                              value = 2, inplace = True)
tabela["Tempo de moradia neste domicílio."].replace(regex=['01','1', 'um'],
                                                              value = 1, inplace = True)
#############

# agrupando dados (tempo de moradia)

tabela["Tempo de moradia neste domicílio."]=pd.cut(
    tabela["Tempo de moradia neste domicílio."],
    bins=[0, 0.9, 5, 10, 20, sys.maxsize],
    labels=['Menos de 1 ano', 'Entre 1 e 5 anos', 'Entre 6 e 10 anos', 'Entre 11 e 20 anos', 'Mais de 20 anos']
 )


# assuntos de jornal


tabela["Se você lê jornal, quais os assuntos que mais lê?"].replace(regex=['Notícias locais', 'Notícias nacionais','Lazer', 'arte e cultura',
                                            'Esporte', 'Notícias policiais','Notícias internacionais','Sociais'],value = 2,  inplace = True)
tabela["Se você lê jornal, quais os assuntos que mais lê?"] = tabela["Se você lê jornal, quais os assuntos que mais lê?"].astype('string')
tabela["Se você lê jornal, quais os assuntos que mais lê?"].replace(regex='2',value ='Notícias Específicas',  inplace = True)                                                                                                         


##########


#display(tabela)

#display(tabela.info())


#################

#Criando os gráficos

respostas = ''

# loop faz todos os gráficos por coluna da tabela de dados
for coluna in tabela:
  if (coluna != 'Escreva algumas linhas sobre sua história e seus sonhos de vida'):
    qtd = tabela[coluna].value_counts()  #conta os dados da coluna
    if ( coluna == 'Qual a empresa que você está contratado agora?' or coluna == 'Quais fontes de entretenimento cultural você usa?') :
      # grafico de barra
       grafico = px.bar(y = qtd.index,
                        x= qtd.values, orientation='h', width=1000, height=1000,
              title=coluna, color_discrete_sequence=px.colors.sequential.RdBu)
       grafico.update_traces(textposition='inside',texttemplate='%{x}')
       grafico.update_traces(hovertemplate = "%{label} <br>N° de Alunos: %{value}")
    else:
      # faz o gráfico de pizza usando os dados contados
      grafico = px.pie(names = qtd.index,
                        values = qtd.values,
                title=coluna, color_discrete_sequence=px.colors.sequential.RdBu) 
      # algumas formatações do gráfico
      grafico.update_traces(hovertemplate = "%{label} <br>N° de Alunos: %{value}", textfont_size=20,
                    title_font_color='#000000', title_font_size=20)
    # mostra gráfico
    display(grafico)
  else :
    # coluna pergunta aberta
    print ('''\n Para vizualização das respostas da questão: ESCREVA ALGUMAS LINHAS SOBRE SUA HISTÓRIA E SEUS SONHOS DE VIDA ,
em formato de nuvem de palavras ,copie o texto abaixo e insira na página https://www.wordclouds.com/   \n 
---- TEXTO ----- :''')
    for linha in tabela['Escreva algumas linhas sobre sua história e seus sonhos de vida']:
      if(linha != 'Preferiu não responder'):
        respostas += linha + '\n'
    
    display(respostas)



Insira o nome do arquivo de dados (inclui .xlsx) : c.xlsx



 Para vizualização das respostas da questão: ESCREVA ALGUMAS LINHAS SOBRE SUA HISTÓRIA E SEUS SONHOS DE VIDA ,
em formato de nuvem de palavras ,copie o texto abaixo e insira na página https://www.wordclouds.com/   
 
---- TEXTO ----- :


'Em 2020 comecei a estudar programação sozinho em casa, em 2021 entrei pra FATEC e conquistei uma vaga de estagio. Espero que futuramente consiga um bom salario.\nConhecer o Brasil \nMudar de pais\nDesde novo utilizo computador, criei um servidor de MuOnline no qual já está online a + de 4 anos, e amo essa aréa\nDesejo me formar na área sendo muito bem capacitado e conseguir um bom emprego para realizar meus sonhos mundo a fora.\n...\nter uma vida tranquila \nSempre me identifiquei na área da tecnologia e tive a oportunidade de estudar na área e espero conquistar um emprego bom \nEspero me profissionalizar na área e obter novos conhecimentos\nEntrei na Fatec com o objetivo de me capacitar profissionalmente e espero conquistar uma vaga no mercado de trabalho como programador. \nFicar bilionário\nPretendo me tornar um ótimo profissional da area e melhorar minha renda.\n.\nMeu nome é Guilherme Vinaud, tenho 30 anos e me interessei pela área de TI depois que fiz um curso na Etec. Não traba